In [1]:
%load_ext autoreload
%autoreload 2

In [21]:
import os
if "notebooks" in os.getcwd():
    os.chdir("..")

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from backtesting import Backtest, Strategy

In [3]:
prices_df = yf.download(
    tickers = "USDBRL=X",
    period = "60d",
    interval = "5m",
    prepost = False
)

[*********************100%***********************]  1 of 1 completed


## Validation-Test split
EXPLAIN

In [5]:
prices_df['date'] = prices_df.index.date

In [10]:
end_validation_date = prices_df['date'].quantile(0.6)
start_test_date = prices_df['date'].quantile(0.7)

In [12]:
end_validation_date, start_test_date

(datetime.date(2023, 1, 4), datetime.date(2023, 1, 11))

In [15]:
train_df = prices_df[prices_df['date'] <= end_validation_date]

In [16]:
train_df

,Open,High,Low,Close,Adj Close,Volume,date
Datetime,,,,,,,
2022-11-15 00:00:00+00:00,5.3132,5.3321,5.3132,5.3321,5.3321,0,2022-11-15
2022-11-15 00:10:00+00:00,5.3321,5.3321,5.3321,5.3321,5.3321,0,2022-11-15
2022-11-15 00:15:00+00:00,5.3321,5.3321,5.3321,5.3321,5.3321,0,2022-11-15
2022-11-15 00:25:00+00:00,5.3321,5.3321,5.3132,5.3132,5.3132,0,2022-11-15
2022-11-15 00:30:00+00:00,5.3132,5.3321,5.3132,5.3132,5.3132,0,2022-11-15
...,...,...,...,...,...,...,...
2023-01-04 23:15:00+00:00,5.4441,5.4441,5.4441,5.4441,5.4441,0,2023-01-04
2023-01-04 23:25:00+00:00,5.4441,5.4441,5.4441,5.4441,5.4441,0,2023-01-04
2023-01-04 23:45:00+00:00,5.4441,5.4441,5.4441,5.4441,5.4441,0,2023-01-04


In [19]:
test_df = prices_df[prices_df['date']>= start_test_date]

In [20]:
test_df

,Open,High,Low,Close,Adj Close,Volume,date
Datetime,,,,,,,
2023-01-11 00:00:00+00:00,5.2096,5.2096,5.2096,5.2096,5.2096,0,2023-01-11
2023-01-11 00:05:00+00:00,5.2096,5.2096,5.2096,5.2096,5.2096,0,2023-01-11
2023-01-11 00:10:00+00:00,5.2096,5.2096,5.2096,5.2096,5.2096,0,2023-01-11
2023-01-11 00:30:00+00:00,5.2096,5.2096,5.2096,5.2096,5.2096,0,2023-01-11
2023-01-11 00:35:00+00:00,5.2096,5.2096,5.2096,5.2096,5.2096,0,2023-01-11
...,...,...,...,...,...,...,...
2023-02-06 21:05:00+00:00,5.1520,5.1520,5.1467,5.1499,5.1499,0,2023-02-06
2023-02-06 21:10:00+00:00,5.1505,5.1505,5.1432,5.1445,5.1445,0,2023-02-06
2023-02-06 21:15:00+00:00,5.1437,5.1465,5.1427,5.1427,5.1427,0,2023-02-06


## Create strategy